# 1. 添加数据字典知识

In [38]:
import openai
import os
import numpy as np
import pandas as pd
import json
import io
from openai import OpenAI
import inspect
import pymysql

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai.api_key)

In [62]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="gpt-3.5-turbo",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_function_description=json.loads(response.choices[0].message.content.replace("```","").replace("json",""))
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [63]:
def run_conversation(messages, functions_list=None, model="gpt-3.5-turbo"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-3.5-turbo
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message

        tool_calls = response_message.tool_calls
        if tool_calls:
            messages.append(response_message) 
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            ) 
            # 获取最终结果
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [64]:
def get_user_demographics(sql_query):
    """
    用户获取user_demographics 表的相关信息
    :param sql_query: 字符串形式的SQL语句
    :return SQL查询的user_demographics 表的相关信息
    """
    mysql_pw="iquery_agent"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='iquery_agent',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='iquery',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")         

In [65]:
functions_list = [get_user_demographics]

In [69]:
tools = auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'get_user_demographics',
   'description': '用户获取user_demographics 表的相关信息',
   'parameters': {'type': 'object',
    'properties': {'sql_query': {'type': 'string',
      'description': '字符串形式的SQL语句'}},
    'required': ['sql_query']}}}]

In [70]:
 messages=[
            {"role": "user", "content": "请问user_demographics表中个人属性为老年男性的数据总共有多少条？"}
        ]
    
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )

response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5VK3ywSsn76wNBfAEc17FaWr', function=Function(arguments='{"sql_query":"SELECT COUNT(*) FROM user_demographics WHERE age >= 60 AND gender = \'male\'"}', name='get_user_demographics'), type='function')])

In [71]:
# 打开并读取Markdown文件
with open('/root/autodl-tmp/iquery项目/data/数据字典/iquery数据字典.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
md_content

'# iquery数据字典：iquery数据库数据字典\n\n本数据字典记录了iquery数据库中各张数据表的基本情况。\n\n## 1.user_demographics数据表\n\n- 基本解释\n\n  \u200b\t\tuser_demographics数据表记录了电信用户的个人基本情况，主要涵盖客户基本生物属性，包括性别、年龄状况、是否结婚以及是否经济独立等。\n\n- 数据来源\n\n  \u200b\tuser_demographics数据集由一线业务人员人工采集记录，并且通过回访确认相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| customerID | 客户ID，user_demographics数据表主键 |              | 由数字和字母组成的 | VARCHAR(255) |\n| gender | 用户的性别 | Female, Male | Female (女性), Male (男性) | VARCHAR(255) |\n| SeniorCitizen | 是否为老人 | 0, 1 | 0 (不是), 1 (是) | INT |\n| Partner | 用户是否有伴侣 | Yes, No | Yes (有), No (没有) | VARCHAR(255) |\n| Dependents | 用户经济是否独立，往往用于判断用户是否已经成年 | No, Yes | Yes (有), No (没有) | VARCHAR(255) |\n\n## 2.user_services数据表\n\n- 基本解释\n\n  \u200b        user_services数据表记录了每位用户订购电信服务的基本情况，截至目前，电信服务商提供了种类多样的服务，包括电话类服务和网络类服务等，本数据集表记录了每位用户订阅电信服务的基本情况。\n\n- 数据来源\n\n  \u200b\t\tuser_services数据表由后台系统

In [72]:
messages=[
      {"role": "system", "content": md_content}, 
      {"role": "user", "content": "请问user_demographics表中个人属性为老年男性的数据总共有多少条？"}
]
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )

response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_e4LEDDcytWdGUx0m98plDpUa', function=Function(arguments='{"sql_query":"SELECT * FROM user_demographics WHERE SeniorCitizen = 1 AND gender = \'Male\'"}', name='get_user_demographics'), type='function')])

In [73]:
messages=[
      {"role": "system", "content": md_content}, 
      {"role": "user", "content": "请帮我介绍下iquery这个数据库的基本情况"}
]
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

response.choices[0].message.content

'iquery数据库是一个用于电信用户数据分析的数据库，主要包括了四张数据表：user_demographics、user_services、user_payments和user_churn。这些数据表记录了电信用户的个人基本情况、订购的电信服务情况、支付情况以及流失情况。用户可以通过iquery数据库中的数据来进行用户行为分析、用户流失预测、产品定价优化等工作。每张数据表都有各自的字段说明，方便用户了解和利用数据库中的数据。'

In [74]:
messages=[
      {"role": "system", "content": md_content}, 
      {"role": "user", "content": "user_demographics数据表和user_services数据表哪张表的数据可信度更高呢？"}
]
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

print(response.choices[0].message.content)

根据描述，user_demographics数据表由一线业务人员人工采集记录，并通过回访确认相关信息，数据集的准确性和可信度都非常高。而user_services数据表由后台系统自动生成，并未经过人工校验。因此，user_demographics数据表的数据可信度更高。


# 2. 实现代码执行需要确认逻辑

In [75]:
def sql_inter(sql_query):
    """
    用于获取iquery数据库中各张表的有关相关信息，\
    核心功能是将输入的SQL代码传输至iquery数据库所在的MySQL环境中进行运行，\
    并最终返回SQL代码运行结果。需要注意的是，本函数是借助pymysql来连接MySQL数据库。
    :param sql_query: 字符串形式的SQL查询语句，用于执行对MySQL中iquery数据库中各张表进行查询，并获得各表中的各类相关信息
    :return：sql_query在MySQL中的运行结果。
    """
    
    mysql_pw = "iquery_agent"
    
    connection = pymysql.connect(
            host='localhost',  # 数据库地址
            user='iquery_agent',  # 数据库用户名
            passwd=mysql_pw,  # 数据库密码
            db='iquery',  # 数据库名
            charset='utf8'  # 字符集选择utf8
        )
    
    try:
        with connection.cursor() as cursor:
            # SQL查询语句
            sql = sql_query
            cursor.execute(sql)

            # 获取查询结果
            results = cursor.fetchall()

    finally:
        connection.close()
    
    
    return json.dumps(results)

In [76]:
functions_list = [sql_inter]

In [97]:
tools = auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'sql_inter',
   'description': '用于获取iquery数据库中各张表的有关相关信息，核心功能是将输入的SQL代码传输至iquery数据库所在的MySQL环境中进行运行，并最终返回SQL代码运行结果。需要注意的是，本函数是借助pymysql来连接MySQL数据库。',
   'parameters': {'type': 'object',
    'properties': {'sql_query': {'type': 'string',
      'description': '字符串形式的SQL查询语句，用于执行对MySQL中iquery数据库中各张表进行查询，并获得各表中的各类相关信息'}},
    'required': ['sql_query']}}}]

In [ ]:
messages = [
    {"role": "system", "content": md_content},
    {"role": "user", "content": "请问user_demographics数据表的主键和user_services数据表的主键是否完全一致？"}
]
run_conversation(messages, functions_list=functions_list, model="gpt-3.5-turbo")

'user_demographics数据表的主键为"0003-MKNFE"，而user_services数据表的主键为"0002-ORFBO"，所以它们并不完全一致。'

In [91]:
def extract_sql(json_str):
    # 提取并返回'sql_query'的值
    return json_str.get('sql_query', None)


In [101]:
def check_code_run(messages, functions_list=None, model="gpt-3.5-turbo",auto_run = True):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-3.5-turbo
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)
        

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        if tool_calls:
            messages.append(response_message) 
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
              
              
                if auto_run == False:
                    sql_query = extract_sql(function_args)

                    res = input('即将执行以下代码：%s。是否确认并继续执行（1），或者退出本次运行过程（2）' % sql_query)
                    if res == '2':
                        print("终止运行")
                        return None
                    else:
                        print("正在执行代码，请稍后...")
                        
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            ) 
            # 获取最终结果
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
    del messages
                
    return final_response

In [111]:
messages = [
    {"role": "system", "content": md_content},
    {"role": "user", "content": "请问iquery数据库下user_demographics表的第10条数据内容是？"}
]

In [112]:
functions_list

[<function __main__.sql_inter(sql_query)>]

In [113]:
check_code_run(messages, 
               functions_list=functions_list, 
               model="gpt-3.5-turbo", 
               auto_run = False)

即将执行以下代码：SELECT * FROM user_demographics LIMIT 9,1。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...


'user_demographics表的第10条数据内容如下：\n\n| customerID | gender | SeniorCitizen | Partner | Dependents |\n|------------|--------|---------------|---------|------------|\n| 0019-GFNTW | Female | 0 | No | No |'

In [115]:
messages = [
    {"role": "system", "content": md_content},
    {"role": "user", "content": "请问iquery数据库下user_demographics表有多少条数据？"}
]
check_code_run(messages, 
               functions_list=functions_list, 
               model="gpt-3.5-turbo", 
               auto_run = False)

即将执行以下代码：SELECT COUNT(*) FROM user_demographics。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...


'user_demographics表下共有5353条数据。'

In [117]:
messages = [
    {"role": "system", "content": md_content},
    {"role": "user", "content": "请问iquery数据库下user_demographics表中，男性和女性的分别有多少人"}
]
check_code_run(messages, 
               functions_list=functions_list, 
               model="gpt-3.5-turbo", 
               auto_run = False)

发生错误： Expecting value: line 1 column 1 (char 0)
正在重新运行...


即将执行以下代码：SELECT gender, COUNT(*) AS count FROM user_demographics GROUP BY gender。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...


'在user_demographics表中，男性有2744人，女性有2609人。'

# 3. 实现多轮对话效果

In [118]:
import tiktoken

def chat_with_inter(functions_list=None, 
                    prompt="你好呀", 
                    model="gpt-3.5-turbo", 
                    system_message=[{"role": "system", "content": "你是一个智能助手。"}], 
                    auto_run = True):
    
    # 创建函数列表对应的参数解释列表
    functions = auto_functions(functions_list)
    
    # 多轮对话阈值
    if 'gpt-4' in model:
        tokens_thr = 6000
    elif '16k' in model:
        tokens_thr = 14000
    else:
        tokens_thr = 3000
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    ## 计算token大小
    embedding_model = "text-embedding-ada-002"
    # 模型对应的分词器（TOKENIZER）
    embedding_encoding = "cl100k_base"
    encoding = tiktoken.get_encoding(embedding_encoding)
    tokens_count = len(encoding.encode((prompt + system_message[0]["content"])))
    
    while True:           
        answer = check_code_run(messages, 
                                functions_list=functions_list, 
                                model=model, 
                                auto_run = auto_run)
        
        
        print(f"模型回答: {answer}")

        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            del messages
            break

        # 记录新一轮问答
        messages.append({"role": "assistant", "content": answer})
        messages.append({"role": "user", "content": user_input})
        
        # 计算当前总token数
        tokens_count += len(encoding.encode((answer + user_input)))
        
        # 删除超出token阈值的对话内容
        while tokens_count >= tokens_thr:
            tokens_count -= len(encoding.encode(messages.pop(1)["content"]))

In [119]:
chat_with_inter(functions_list=functions_list, 
                prompt="我想根据iquery数据库中数据分析用户流失情况，请问需要用到iquery数据库中的哪几张表呢？", 
                model="gpt-3.5-turbo", 
                system_message=[{"role": "system", "content": md_content}], 
                auto_run = False)

即将执行以下代码：SHOW TABLES;。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...
模型回答: 要分析用户流失情况，你需要用到iquery数据库中的user_churn数据表以及user_demographics数据表。这两张数据表可以提供用户当前的流失状态以及用户的个人基本情况，从而进行流失分析。


您还有其他问题吗？(输入退出以结束对话):  请问user_demographics表的第10条数据内容是？
即将执行以下代码：SELECT * FROM user_demographics LIMIT 9,1;。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...
模型回答: user_demographics表的第10条数据内容如下：

| customerID | gender | SeniorCitizen | Partner | Dependents |
|------------|--------|---------------|---------|------------|
| 0019-GFNTW | Female | 0 | No | No |


您还有其他问题吗？(输入退出以结束对话):  请问user_demographics表中一共有多少条数据？
即将执行以下代码：SELECT COUNT(*) FROM user_demographics;。是否确认并继续执行（1），或者退出本次运行过程（2） 1


正在执行代码，请稍后...
模型回答: user_demographics表中共有5353条数据。


您还有其他问题吗？(输入退出以结束对话):  退出
